This workbook is meant to use beautiful soup to scrape wedding venue prices in Los Angeles, California, USA from Wedding Wire.

In [27]:
#import BeautifulSoup, pandas, requests, json
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd

#initialize list of wedding page loads
wedding_pages = []

#initialize dict to hold vendor details
vendor_dict = {}
#initialize list to hold list of dicts and facilitate DataFrame conversion
rows = []

#identify number of pages to scrape from site
num_pages = 6

In [28]:
#pull first page html and append to the list of wedding pages, this solves for the inconsistent first URL
wedding_pg = requests.get("https://www.weddingwire.com/shared/search?group_id=1&region_id=10095&state_id=404")
soup = BeautifulSoup(wedding_pg.text, 'html.parser')
wedding_pages.append(soup)

#loop through the remaining pages to pull in and append html to the list of wedding pages using the consistent URL page structure
for i in range(2, num_pages+1):
    wedding_pgs = requests.get("https://www.weddingwire.com/shared/search?group_id=1&region_id=10095&state_id=404&page="+str(i))
    soup = BeautifulSoup(wedding_pgs.text, 'html.parser')
    wedding_pages.append(soup)

In [29]:
#create a list of all li items from bs4 objects with vendor data
#this loop accesses and iterates over all bs4 objects stored in the wedding_pages list
for soup in wedding_pages:
    vendor_list = soup.find_all("li", attrs = {'aria-label':"Vendor"})
    
#iterate over list of vendor li objects. Add name and desired attributes to vendor_dict 
#nested for loop will add values to vendor_dict as they are identified in the page
    for vendor in vendor_list:
        vendor_name = vendor["data-ec-name"]
        vendor_info_json = vendor['data-vendor-info']
        vendor_info = json.loads(vendor_info_json)
        vendor_dict[vendor_name] = vendor_info

In [30]:
#extract values of interest from vendor dictionary using dict.get method, assign values to variables
for vendor_name, vendor_info in vendor_dict.items():
    price = vendor_info.get('price') 
    region = vendor_info.get('region')
    country = vendor_info.get('country')
    postal_code = vendor_info.get('address', {}).get('postal_code')
    sector = vendor_info.get('sector')

#create a dictionary of the values of interest above
    row = {
        'vendor name': vendor_name,
        'price': price,
        'region': region,
        'country': country,
        'postal code': postal_code,
        'sector': sector
    }
#append dictionary of values to the rows list for conversion to dataframe
    rows.append(row)

In [31]:
#convert vendor data to DataFrame
vendor_df = pd.DataFrame(rows)

In [32]:
#print df values for confirmation
vendor_df

,vendor name,price,region,country,postal code,sector
0,The Malibu Garden Estate,1.00,Los Angeles,US,90265,Mansion Weddings
1,Metropol Banquet,1.00,Los Angeles,US,91204,Banquet Halls
2,Hart & Main,"10,656.00",Los Angeles,US,91321,Banquet Halls
3,Quiet Cannon,"6,985.00",Los Angeles,US,90640,Banquet Halls
4,Pomona Valley Mining Company,"7,401.00",Los Angeles,US,91768,Banquet Halls
...,...,...,...,...,...,...
182,Cal Aero Events,1.00,Inland Empire,US,91710,Country Club Weddings
183,Classic Club,"12,304.00",Inland Empire,US,92211,Country Club Weddings
184,NoHo First UMC,1.00,Los Angeles,US,91601,Church & Temple Weddings
185,AOC Wine Bar & Restaurant,1.00,Los Angeles,US,90048,Rehearsal Dinner


In [33]:
#create a csv file with extracted data
vendor_df.to_csv("la_wedding_venue_data.csv", index=False)